In [176]:
import pandas as pd
import os
import pathlib
import zipfile

In [177]:
home = os.getcwd() + "\\Data"
root = home + "\\Data for Proposed Iron Ore Deposits"
root

'D:\\Documents\\Masters\\IITGNX - Energy Policy and Regulation\\Capstone Project\\Final Simulation\\Data\\Data for Proposed Iron Ore Deposits'

In [178]:
home

'D:\\Documents\\Masters\\IITGNX - Energy Policy and Regulation\\Capstone Project\\Final Simulation\\Data'

In [180]:
len(os.listdir(root))

36

<h3> Caculate Location-wise RE Characteristics </h3>

In [181]:
def calculate_re_features(file_solar_dni, file_mean_wind_speed, file_wind_heatmap, file_wind_power_curve):
    df_solar = pd.read_excel(file_solar_dni, sheet_name="Hourly_profiles", engine="openpyxl", skiprows=4)
    df_solar.rename(columns={"Unnamed: 0": "Hour"}, inplace=True)
    df_solar = df_solar[df_solar["Hour"]!="Sum"]
    month_columns = [col for col in df_solar.columns if col!="Hour"]
    dni_list = [item[0] for item in list(df_solar[month_columns].values.reshape(-1,1))]
    cf_list = [d/1000 for d in dni_list]
    df_cf = pd.DataFrame({"cf":cf_list})
    df_cf["cf"].describe()
    
    df_power_curve = pd.read_excel(file_wind_power_curve, engine="openpyxl")
    
    exceptions = None
    try:
        df_mean_wind_speed = pd.read_csv(file_mean_wind_speed)
        df_heatmap_data = pd.read_csv(file_wind_heatmap)
    except Exception as e:
       exceptions = e
       return None, exceptions
    
    ten_year_mean_wind_speed = df_mean_wind_speed["val"].mean()
    df_heatmap_data.rename(columns={"Unnamed: 0":"Hour"}, inplace=True)
    month_columns = [col for col in df_heatmap_data.columns if col!="Hour"]
    wsi_list = [item[0] for item in df_heatmap_data[month_columns].values.reshape(-1,1)]
    # len(wsi_list)
    ws_list = [ten_year_mean_wind_speed*w for w in wsi_list]
    wp_list = [np.interp(ws, df_power_curve["u (m/s)"].values, df_power_curve["P (kW)"].values) for ws in ws_list]
    wcf_list = [wp/3450 for wp in wp_list]
    df_wcf = pd.DataFrame({"wcf":wcf_list})
    
    
    
    df_output = pd.DataFrame({
                               'CoV hourly solar CF'     :[df_cf.cov().values[0][0]],
                               'CoV hourly wind CF'      :[df_wcf.cov().values[0][0]],
                               'median hourly solar CF'  :[df_cf.median().values[0]],
                               'median hourly wind CF'   :[df_wcf.median().values[0]]
                               })
    return df_output, exceptions

In [182]:
file_wind_power_curve = root + "\\Wind Power Curve.xlsx"

for location in [folder for folder in os.listdir(root) if "xlsx" not in folder.lower()]:
    location_path = root+"\\"+location
    for file in os.listdir(location_path):
        if "GSA" in file and ".xlsx" in file.lower():
            file_solar_dni = location_path+"\\"+file
    print(file_solar_dni)
    for item in os.listdir(location_path):
        if ".zip" in item.lower():
            path_to_zip_file = location_path+"\\"+item
    dir_to_extract_to = location_path+"\\GWA"
    try:
        os.mkdir(dir_to_extract_to)
    except:
        pass
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(dir_to_extract_to)
    print(path_to_zip_file)
    file_mean_wind_speed = dir_to_extract_to + "\\windSpeed.csv"
    file_wind_heatmap    = dir_to_extract_to + "\\heatmapData.csv"
    
    df_output, exceptions = calculate_re_features(file_solar_dni, file_mean_wind_speed, file_wind_heatmap, file_wind_power_curve)
    if df_output is None:
        print(exceptions)
    else:
        try:
            output_dir = location_path+"\\Output"
            os.mkdir(output_dir)
            df_output.to_excel(output_dir+"\\output.xlsx", engine="openpyxl", index=False)
        except:
            pass
    


D:\Documents\Masters\IITGNX - Energy Policy and Regulation\Capstone Project\Final Simulation\Data\Data for Proposed Iron Ore Deposits\Aethel Mining Cabeco da Mua Mine\GSA_Report_Ecopista do Sabor, 5160-085 Torre de Moncorvo, Portugal.xlsx
D:\Documents\Masters\IITGNX - Energy Policy and Regulation\Capstone Project\Final Simulation\Data\Data for Proposed Iron Ore Deposits\Aethel Mining Cabeco da Mua Mine\gwa-plot-data_2025-05-07174664139582506_09_55_UTC.zip
D:\Documents\Masters\IITGNX - Energy Policy and Regulation\Capstone Project\Final Simulation\Data\Data for Proposed Iron Ore Deposits\Ansteel Heishilizi Mine\GSA_Report_Liaoning.xlsx
D:\Documents\Masters\IITGNX - Energy Policy and Regulation\Capstone Project\Final Simulation\Data\Data for Proposed Iron Ore Deposits\Ansteel Heishilizi Mine\gwa-plot-data_2025-05-04174633226612904_17_46_UTC.zip
D:\Documents\Masters\IITGNX - Energy Policy and Regulation\Capstone Project\Final Simulation\Data\Data for Proposed Iron Ore Deposits\Austrominer

In [185]:
exceptions

<h3> Consolidate RE Characteristics of All Locations </h3>

In [183]:
df_re_list = []
for location in [folder for folder in os.listdir(root) if "xlsx" not in folder.lower()]:
    output_path = root+"\\"+location+"\\Output\output.xlsx"
    print(output_path)
    try:
        df_re = pd.read_excel(output_path, engine="openpyxl")
        df_re["Asset name (English)"] = location
        df_re_list.append(df_re)
    except:
        pass
    
len(df_re_list)

df_final = pd.concat(df_re_list)
df_final.reset_index(inplace=True)
df_final.drop(columns=["index"], inplace=True)
df_final

D:\Documents\Masters\IITGNX - Energy Policy and Regulation\Capstone Project\Final Simulation\Data\Data for Proposed Iron Ore Deposits\Aethel Mining Cabeco da Mua Mine\Output\output.xlsx
D:\Documents\Masters\IITGNX - Energy Policy and Regulation\Capstone Project\Final Simulation\Data\Data for Proposed Iron Ore Deposits\Ansteel Heishilizi Mine\Output\output.xlsx
D:\Documents\Masters\IITGNX - Energy Policy and Regulation\Capstone Project\Final Simulation\Data\Data for Proposed Iron Ore Deposits\Austromineral Dommel Nissar Mine\Output\output.xlsx
D:\Documents\Masters\IITGNX - Energy Policy and Regulation\Capstone Project\Final Simulation\Data\Data for Proposed Iron Ore Deposits\Baiyuli Mine\Output\output.xlsx
D:\Documents\Masters\IITGNX - Energy Policy and Regulation\Capstone Project\Final Simulation\Data\Data for Proposed Iron Ore Deposits\Baobab Steel Tete Mine\Output\output.xlsx
D:\Documents\Masters\IITGNX - Energy Policy and Regulation\Capstone Project\Final Simulation\Data\Data for Pr

,CoV hourly solar CF,CoV hourly wind CF,median hourly solar CF,median hourly wind CF,Asset name (English)
0,0.064198,0.012887,0.000,0.204997,Aethel Mining Cabeco da Mua Mine
1,0.035901,0.040155,0.000,0.437042,Ansteel Heishilizi Mine
2,0.060330,0.155758,0.000,0.448596,Austromineral Dommel Nissar Mine
3,0.050395,0.026423,0.000,0.253284,Baiyuli Mine
4,0.036481,0.034352,0.000,0.349368,Bensteel Hualinggou Mine
5,0.082163,0.017056,0.039,0.195600,Beren Mining Khadat Tolgoi Mine
6,0.079495,0.011561,0.000,0.209384,Beren Mining Tsenkher Mine
7,0.056769,0.015832,0.000,0.096570,Darkhan Hust-ul Mine
8,0.047099,0.017741,0.031,0.374605,Grange Southdown Mine
9,0.110167,0.061564,0.000,0.517935,Hajigak Mine


In [186]:
home

'D:\\Documents\\Masters\\IITGNX - Energy Policy and Regulation\\Capstone Project\\Final Simulation\\Data'

In [187]:
df_iron_ore_deposits = pd.read_excel(home+"\\IronOre_Deposits_Proposed.xlsx", engine="openpyxl")

In [188]:
df_iron_ore_deposits.head()

,n,Asset name (English),Asset name (other language),Coordinates,Coordinate accuracy,Municipality,Subnational unit,Country/Area,Region,Production 2023 (ttpa),...,"Total resource (inferred, indicated and measured, thousand metric tonnes)",Operating status,Start date,Stop date,Owner,Owner GEM Entity ID,Owner name in local language/script,Parent,Parent GEM Entity ID,GEM wiki page URL
0,P100000128013,Hajigak Mine,NaN,"34.667981, 68.062676",exact,Hajigak,Bamyan,Afghanistan,Asia Pacific,NaN,...,1769900,proposed,unknown,NaN,unknown,E100000132388,None,--,--,https://gem.wiki/Hajigak_Mine
1,P100000128380,CZR Resources Robe Mesa Mine,NaN,"-22.452537, 116.716311",approximate,Nanutarra,Western Australia,Australia,Asia Pacific,NaN,...,45200,proposed,2025,NaN,CZR Resources Ltd [85.0%]; Creasy Group Pty Lt...,E100002002780 [85.0%]; E100002002852 [15.0%],NaN,CZR Resources Ltd [85.0%]; Creasy Group Pty Lt...,E100002002780 [85.0%]; E100002002852 [15.0%],https://gem.wiki/CZR_Resources_Robe_Mesa_Mine
2,P100000129284,Grange Southdown Mine,NaN,"-34.496557, 118.695495",approximate,Wellstead,Western Australia,Australia,Asia Pacific,NaN,...,1200000,proposed,2026,NaN,Southdown Joint Venture [100.0%],E100002004410 [100.0%],NaN,Grange Resources Ltd [70.0%]; SRT Australia Pt...,E100002002889 [70.0%]; E100002004411 [30.0%],https://gem.wiki/Grange_Southdown_Mine
3,P100000128106,Rio Tinto Western Range Mine,NaN,"-23.221489, 117.586751",exact,Innawanga,Western Australia,Australia,Asia Pacific,NaN,...,397000,proposed,2025,NaN,Rio Tinto Ltd [54.0%]; China Baowu Steel Group...,E100002002847 [54.0%]; E100001000472 [46.0%],None,Rio Tinto Ltd [54.0%]; China Baowu Steel Group...,E100002002847 [54.0%]; E100001000472 [46.0%],https://gem.wiki/Rio_Tinto_Western_Range_Mine
4,P100000128189,Howse Mine,NaN,"54.890153, -67.080217",exact,Subd. D,Newfoundland and Labrador,Canada,North America,NaN,...,116000,proposed,unknown,NaN,Howse Minerals Ltd [100.0%],E100002003876 [100.0%],None,--,--,https://gem.wiki/Howse_Mine


In [189]:
df_final = df_iron_ore_deposits.merge(df_final, on="Asset name (English)", how="inner")
df_final.head()

,n,Asset name (English),Asset name (other language),Coordinates,Coordinate accuracy,Municipality,Subnational unit,Country/Area,Region,Production 2023 (ttpa),...,Owner,Owner GEM Entity ID,Owner name in local language/script,Parent,Parent GEM Entity ID,GEM wiki page URL,CoV hourly solar CF,CoV hourly wind CF,median hourly solar CF,median hourly wind CF
0,P100000128013,Hajigak Mine,NaN,"34.667981, 68.062676",exact,Hajigak,Bamyan,Afghanistan,Asia Pacific,NaN,...,unknown,E100000132388,None,--,--,https://gem.wiki/Hajigak_Mine,0.110167,0.061564,0.000,0.517935
1,P100000129284,Grange Southdown Mine,NaN,"-34.496557, 118.695495",approximate,Wellstead,Western Australia,Australia,Asia Pacific,NaN,...,Southdown Joint Venture [100.0%],E100002004410 [100.0%],NaN,Grange Resources Ltd [70.0%]; SRT Australia Pt...,E100002002889 [70.0%]; E100002004411 [30.0%],https://gem.wiki/Grange_Southdown_Mine,0.047099,0.017741,0.031,0.374605
2,P100000128106,Rio Tinto Western Range Mine,NaN,"-23.221489, 117.586751",exact,Innawanga,Western Australia,Australia,Asia Pacific,NaN,...,Rio Tinto Ltd [54.0%]; China Baowu Steel Group...,E100002002847 [54.0%]; E100001000472 [46.0%],None,Rio Tinto Ltd [54.0%]; China Baowu Steel Group...,E100002002847 [54.0%]; E100001000472 [46.0%],https://gem.wiki/Rio_Tinto_Western_Range_Mine,0.133242,0.011659,0.000,0.202830
3,P100000128158,Ansteel Heishilizi Mine,黑石砬子(Chinese),"40.974935, 123.000169",exact,Anshan,Liaoning,China,Asia Pacific,NaN,...,Ansteel Group Co Ltd [100.0%],E100000124667 [100.0%],鞍钢集团有限公司,Ansteel Group Co Ltd [100.0%],E100000124667 [100.0%],https://gem.wiki/Ansteel_Heishilizi_Mine,0.035901,0.040155,0.000,0.437042
4,P100000128723,Baiyuli Mine,白峪里 铁矿(Chinese),"38.972335, 113.011755",exact,Xinzhou,Shanxi,China,Asia Pacific,NaN,...,Baiyuli Iron Mine in Daixian County [100.0%],E100002002767 [100.0%],代县白峪里铁矿,--,--,https://gem.wiki/Baiyuli_Mine,0.050395,0.026423,0.000,0.253284


In [190]:
df_final

,n,Asset name (English),Asset name (other language),Coordinates,Coordinate accuracy,Municipality,Subnational unit,Country/Area,Region,Production 2023 (ttpa),...,Owner,Owner GEM Entity ID,Owner name in local language/script,Parent,Parent GEM Entity ID,GEM wiki page URL,CoV hourly solar CF,CoV hourly wind CF,median hourly solar CF,median hourly wind CF
0,P100000128013,Hajigak Mine,NaN,"34.667981, 68.062676",exact,Hajigak,Bamyan,Afghanistan,Asia Pacific,NaN,...,unknown,E100000132388,None,--,--,https://gem.wiki/Hajigak_Mine,0.110167,0.061564,0.000,0.517935
1,P100000129284,Grange Southdown Mine,NaN,"-34.496557, 118.695495",approximate,Wellstead,Western Australia,Australia,Asia Pacific,NaN,...,Southdown Joint Venture [100.0%],E100002004410 [100.0%],NaN,Grange Resources Ltd [70.0%]; SRT Australia Pt...,E100002002889 [70.0%]; E100002004411 [30.0%],https://gem.wiki/Grange_Southdown_Mine,0.047099,0.017741,0.031,0.374605
2,P100000128106,Rio Tinto Western Range Mine,NaN,"-23.221489, 117.586751",exact,Innawanga,Western Australia,Australia,Asia Pacific,NaN,...,Rio Tinto Ltd [54.0%]; China Baowu Steel Group...,E100002002847 [54.0%]; E100001000472 [46.0%],None,Rio Tinto Ltd [54.0%]; China Baowu Steel Group...,E100002002847 [54.0%]; E100001000472 [46.0%],https://gem.wiki/Rio_Tinto_Western_Range_Mine,0.133242,0.011659,0.000,0.202830
3,P100000128158,Ansteel Heishilizi Mine,黑石砬子(Chinese),"40.974935, 123.000169",exact,Anshan,Liaoning,China,Asia Pacific,NaN,...,Ansteel Group Co Ltd [100.0%],E100000124667 [100.0%],鞍钢集团有限公司,Ansteel Group Co Ltd [100.0%],E100000124667 [100.0%],https://gem.wiki/Ansteel_Heishilizi_Mine,0.035901,0.040155,0.000,0.437042
4,P100000128723,Baiyuli Mine,白峪里 铁矿(Chinese),"38.972335, 113.011755",exact,Xinzhou,Shanxi,China,Asia Pacific,NaN,...,Baiyuli Iron Mine in Daixian County [100.0%],E100002002767 [100.0%],代县白峪里铁矿,--,--,https://gem.wiki/Baiyuli_Mine,0.050395,0.026423,0.000,0.253284
5,P100000128317,Bensteel Hualinggou Mine,花岭沟铁矿(Chinese),"41.478333, 123.646389",exact,Benxi,Liaoning,China,Asia Pacific,NaN,...,Benxi Iron & Steel Group Co Ltd [100.0%],E100002004353 [100.0%],本溪钢铁(集团),--,--,https://gem.wiki/Bensteel_Hualinggou_Mine,0.036481,0.034352,0.000,0.349368
6,P100000128351,Liangzi Iron Ore Mine,四川省盐源县矿山梁子铁矿(Chinese),"27.614028, 101.880833",exact,Liangshan Yi Autonomous Prefecture,Sichuan,China,Asia Pacific,NaN,...,Yanyuan County Jintie Mining Group Co Ltd [100...,E100002003058 [100.0%],盐源县金铁矿业集团有限责任公司,--,--,https://gem.wiki/Liangzi_Iron_Ore_Mine,0.042147,0.053295,0.000,0.100314
7,P100000128005,Wangniangou Iron Mine,苍山县王埝沟铁矿(Chinese),"34.854166, 117.837500",approximate,Linyi,Shandong,China,Asia Pacific,NaN,...,unknown,E100000132388,None,--,--,https://gem.wiki/Wangniangou_Iron_Mine,0.017748,0.008357,0.004,0.168320
8,P100000128264,Yamansu Mine,雅满苏铁矿(Chinese),"41.885194, 93.877420",exact,Hami,Xinjiang,China,Asia Pacific,NaN,...,Xinjiang Iron and Steel Company Yamansu Mining...,E100002003162 [100.0%],新疆钢铁公司雅满苏矿业,--,--,https://gem.wiki/Yamansu_Mine,0.069111,0.037033,0.000,0.306486
9,P100000128355,Zuolong iron Mine,佐龙铁矿(Chinese),"27.333889, 115.310417",exact,Ji'an,Jiangxi,China,Asia Pacific,NaN,...,Yongfeng County Zuolong Iron Mine Co Ltd [100.0%],E100002003055 [100.0%],永丰县佐龙铁矿有限公司,--,--,https://gem.wiki/Zuolong_iron_Mine,0.013997,0.004848,0.001,0.111583


In [191]:
df_final.to_excel(home+"\\df_re_final.xlsx", index=False, engine="openpyxl")